In [4]:
try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile
import re
import subprocess
import os
import xlsxwriter
import time
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

def createInnkppExcel():
    global inn, kpp
    #millis = int(round(time.time() * 1000))
    workbook = xlsxwriter.Workbook(r"C:\Users\Admin\Desktop\ORG_ALL.xlsx")
    worksheet = workbook.add_worksheet()
    col = 0
    category_list = ['592', '589', '544', '1066', '1050',] 
                     #'548', '601', '598', '725', '604', '727', '728']
    last_row = pd.read_excel(r"C:\Users\Admin\Desktop\ORG_ALL.xlsx")
    row = last_row.shape[0]
    for page in range(1,20)[:1]:
        for category_id in category_list:
            link = f'https://www.rbc.ru/companies/search/?category_id={category_id}&query=&status%5B%5D=ACTIVE&page={page}'
            html = requests.get(link)
            soup = BeautifulSoup(html.text,'html.parser')
            full_name = soup.find_all('a', class_='company-name')
            for company in full_name:
                comp_pattern = re.findall('href=[^>]+', str(company))[0].replace('href=', '')
                comp_link = 'https://www.rbc.ru/'+comp_pattern
                comp_link = comp_link.replace('/"', '')
                html = requests.get(comp_link)
                soup = BeautifulSoup(html.text,'html.parser')
                name_org = soup.find_all('h1', class_='company-headline__title')[0]
                name_org = str(name_org).replace('<h1 class="company-headline__title">', '').replace('</h1>', '')
                requsites = soup.find_all('span', class_='copy-text')
                inn = requsites[2]
                inn = re.findall('data-text="\d+"', str(inn))[0].replace('data-text=', '').replace('"', '')
                kpp = requsites[3]
                kpp = re.findall('data-text="\d+"', str(kpp))[0].replace('data-text=', '').replace('"', '')
                address = requsites[0]
                addr1 = re.findall('data-text=.+', str(address))[0].replace('data-text=', '').replace('"', '')
                addr2 = re.findall('data-toggle=.+', str(addr1))[0].replace('data-text=', '').replace('"', '')
                address = addr1.replace(addr2, '')
                worksheet.write(row, col, name_org)
                worksheet.write(row, col+1, inn)
                worksheet.write(row, col+2, kpp)
                worksheet.write(row, col+3, address)
                row+=1
        workbook.close()
    return 0

# Скачивание компаний с РБК

In [3]:
import re
import subprocess
import os
import time
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import numpy as np
import re

pool = ThreadPool(2)



def get_list_company(link):
    html = requests.get(link)
    soup = BeautifulSoup(html.text,'html.parser')
    return soup.find_all('a', class_='company-name')

def get_company_detail(comp_http):
    comp_link = fr'https://www.rbc.ru' + comp_http.get('href')
    html = requests.get(comp_link)
    soup = BeautifulSoup(html.text,'html.parser')
    name_org = soup.find_all('h1', class_='company-headline__title')[0].get_text()
    requsites = soup.find_all('span', class_='copy-text')
    inn = requsites[2].get_text().strip()
    kpp = requsites[3].get_text().strip()
    address = requsites[0].get_text().strip()
    return [name_org, inn, kpp, address]

### Подготовка списка страниц поиска

In [86]:
html = requests.get(r'https://www.rbc.ru/companies/categories/')
soup = BeautifulSoup(html.text,'html.parser')
categ_list = [re.findall(r'\d+', hr.get('href'))[0] for hr in soup.find_all('a', class_='categories__item')]

pages_list = []
for category_id in categ_list:
    # Ссылку формируем с фильтром Активные, чтобы потом по сервису Dadata получить краткое наименование
    pages = list(f'https://www.rbc.ru/companies/search/?category_id={category_id}&query=&status%5B%5D=ACTIVE&page={page}' 
                 for page in range(1,51)[:])
    pages_list += pages

In [102]:
len(pages_list)

2400

### Скачивание с каждой страницы списка компаний (РБК выдает максимум 50 страниц по 20 компаний)

In [103]:
%%time


full_name = pool.map(get_list_company, pages_list)
# a = np.array(full_name)
# full_name = a.flatten()

CPU times: user 4min 3s, sys: 12.6 s, total: 4min 16s
Wall time: 4min 16s


In [116]:
ff_name = []
for i in full_name:
    for j in i:
        ff_name += [j]

In [106]:
a = np.array(full_name)
len(a.flatten())

2400

### Заходим в карточку компании и вытаскиваем реквизты

In [123]:
%%time
raw = pool.map(get_company_detail, ff_name[:])

CPU times: user 46min 40s, sys: 2min 4s, total: 48min 44s
Wall time: 43min 45s


In [126]:
# Сохраняем DataFrame в excel-файл

df = pd.DataFrame(raw, columns=['NAME', 'INN', 'KPP', 'ADDRESS'])#.to_excel('2_47950.xlsx', index=False)

### Получаем все наименования компании: краткое/полное

In [52]:
# подгружаем скаченные с РБК компании
df1 = pd.read_excel('1_980.xlsx')
df2 = pd.read_excel('2_47950.xlsx')
df = pd.concat([df1, df2])

In [57]:
df.to_excel('../data/raw/companies.xlsx', index=False)

In [17]:
df = pd.read_excel('../data/raw/companies.xlsx', engine='openpyxl')

In [91]:
# процедуры получения данных с Dadata по каждой из компаний из списка РБК
def get_dadata_suggestion(query, resource, apy_key='2e443721385453b3bd84679ce85d4a07a735d9c5', count=30):
    """
    Запрос к dadata
    """
    BASE_URL = "https://suggestions.dadata.ru/suggestions/api/4_1/rs/suggest/{}"
    # API_KEY = '806ef7579701a1cc83665906adef64d72c05d8dc'

    url = BASE_URL.format(resource)
    headers = {"Authorization": "Token {}".format(apy_key), "Content-Type": "application/json"}
    data = {"query": query, "count": count}
    r = requests.post(url, data=json.dumps(data), headers=headers)
    return r.json()



def get_verification_data(inp, apy_key):
    """
    Получение всех совпадений от dadata по запросу ИНН
    """
    orgs_data = []
    result = get_dadata_suggestion(inp, "party", apy_key, count=30)
    for item in result['suggestions']:
        orgs_data_element = {'Name1': None, 'INN': None, 'KPP': None, 'Address': None, 'Full_Name': None,
                             'Short_Name': None,
                             'City': None, 'postal_code': None, 'okato': None, 'oktmo': None, 'okpo': None, 
                             'manager_name': None, 'opf_full': None, 'opf_short': None, 'phones': None}
        for key in item:
            if key == 'value':
                orgs_data_element['Name1'] = item[key]
            if key == 'data':
                orgs_data_element['INN'] = item[key].get('inn')
                orgs_data_element['KPP'] = item['data'].get('KPP', 'IP')
                orgs_data_element['KPP'] = item[key].get('kpp')
                orgs_data_element['Address'] = item[key]['address']['unrestricted_value']
                orgs_data_element['Full_Name'] = item[key]['name']['full_with_opf']
                orgs_data_element['Short_Name'] = item[key]['name']['short_with_opf']
                orgs_data_element['City'] = item[key]['address']['data']['city']
                orgs_data_element['postal_code'] = item[key]['address']['data']['postal_code']
                orgs_data_element['okato'] = item[key].get('okato')
                orgs_data_element['oktmo'] = item[key].get('oktmo')
                orgs_data_element['okpo'] = item[key].get('okpo')
                if item[key].get('management') is not None:
                    orgs_data_element['manager_name'] = item[key]['management']['name'] 
                else:
                    orgs_data_element['manager_name'] = None
                orgs_data_element['phones'] = item[key]['phones']
                orgs_data_element['opf_full'] = item[key]['opf']['full']
                orgs_data_element['opf_short'] = item[key]['opf']['short']
        if orgs_data_element['INN'] == str(inp):    
            orgs_data.append(orgs_data_element)
    return orgs_data


def get_dadata(company_info):
    """
    Запуск получения данных от dadata и
    объединение их в один массив с данными от РБК
    """
    raw = []
    apy_key='8bfbf82611b02577af0cc47f8b0ec64785243586'
    
    try:
        dd_res = get_verification_data(company_info[1], apy_key)
        if dd_res is not None:
            for c in dd_res:
                raw.append([company_info[0],company_info[1], company_info[2], company_info[3],
                         c.get('Full_Name') if c.get('Full_Name') is not None else c.get('Name1'),
                                        c.get('Short_Name') if c.get('Short_Name') is not None else c.get('Name1'),
                                        c.get('INN'),
                                        c.get('KPP'),
                                        c.get('Address'),
                                        c.get('City'),
                                        c.get('postal_code'),
                                        c.get('okato'),
                                        c.get('oktmo'),
                                        c.get('okpo'),
                                        c.get('manager_name'),
                                        c.get('phones'),
                                        c.get('opf_full'),
                                        c.get('opf_short'),
                                   ])
        else:
            raw.append([company_info[0],company_info[1], company_info[2], company_info[3], 
                        None, None, None, None, None, None, None, None, None, None, None, None, None, None])
    except:
        raw.append([company_info[0],company_info[1], company_info[2], company_info[3], 
                        None, None, None, None, None, None, None, None, None, None, None, None, None, None])
    return raw

In [63]:
# Действующие ключи DaData для обхода ограничения по бесплатной лицензии
# max 10 000 запросов в сутки

# 19.05.21: 0170cda3cbdcec736de400b19ba39ddca3d2e3bc
# 19.05.21: ef2558a715d20168d537b9244aba4725b3f3eb10
# 19.05.21: a67619aa89073cfa8f85138e620acb470a2feefd
# прод -- 2e443721385453b3bd84679ce85d4a07a735d9c5
# 19.05.21: aa24430c3c36b7bfe23c0e26830797fb296fbf6d
# 19.05.21: 8bfbf82611b02577af0cc47f8b0ec64785243586
# 04.05.21: fe20dc7811aab14904b93aabb20d62fead71539c

In [ ]:
# тестируем проверку
get_verification_data('7743077630', '8bfbf82611b02577af0cc47f8b0ec64785243586')

In [92]:
%%time

#запускем получение данных с дадаты: указываем интервалы значений  по количеству 
# компаний на один ключ(max 10 000 запросов в сутки)
raww = pool.map(get_dadata, df.values[40000:50000])

Wall time: 8min 3s


In [93]:
raww

df_raw = []
for i in raww:
    for j in i:
        df_raw += [j]

In [94]:
len(df_raw)

9994

In [95]:
pd.DataFrame(df_raw, 
             columns=['RBK_NAME', 'RBK_INN', 'RBK_KPP', 'RBK_ADDR', 'DD_FNAME', 'DD_SNAME', 
                'DD_INN', 'DD_KPP', 'DD_ADDR', 'DD_CITY', 'DD_POSTAL_CODE', 'DD_OKATO'
                , 'DD_OKTMO', 'DD_OKPO', 'DD_MANAGER_NAME', 'DD_PHONES', 'DD_OPF_FULL', 'DD_OPF_SHORT'])

,RBK_NAME,RBK_INN,RBK_KPP,RBK_ADDR,DD_FNAME,DD_SNAME,DD_INN,DD_KPP,DD_ADDR,DD_CITY,DD_POSTAL_CODE,DD_OKATO,DD_OKTMO,DD_OKPO,DD_MANAGER_NAME,DD_PHONES,DD_OPF_FULL,DD_OPF_SHORT
0,ООО СЕРВИС-ИНТЕГРАТОР ОБСЛУЖИВАНИЕ МЕСТОРОЖДЕНИЙ,7710943193,770301001,Г. МОСКВА Ш. ЗВЕНИГОРОДСКОЕ Д. 18/20 К. 1 ЭТАЖ...,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СЕРВ...","ООО ""СИ ОМ""",7710943193,770301001,"123022, г Москва, Пресненский р-н, Звенигородс...",Москва,123022,45286575000,45380000000,17821586,Андрианов Сергей Владимирович,None,Общество с ограниченной ответственностью,ООО
1,ООО АСАН,7727299587,772701001,ГОРОД МОСКВА УЛИЦА ЮЖНОБУТОВСКАЯ ДОМ 117 ПОДВА...,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АСАН""","ООО ""АСАН""",7727299587,772701001,"117042, г Москва, р-н Южное Бутово, ул Южнобут...",Москва,117042,45293594000,45909000000,04462370,Козлов Александр Валерьевич,None,Общество с ограниченной ответственностью,ООО
2,ООО ДЭЙЛИ ФУД,7705899559,770901001,ГОРОД МОСКВА УЛИЦА СТАНИСЛАВСКОГО 21 СТР.1,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДЭЙЛ...","ООО ""ДЭЙЛИ ФУД""",7705899559,770901001,"109004, г Москва, Таганский р-н, ул Станиславс...",Москва,109004,45286580000,45381000000,63671882,Василюк Ольга Никитична,None,Общество с ограниченной ответственностью,ООО
3,ООО ЦЕНТР ПРЕДПРИНИМАТЕЛЬСТВА ДИНАСТИЯ,7713291570,772501001,ГОРОД МОСКВА НАБЕРЕЖНАЯ НАГАТИНСКАЯ ДОМ 10 КОР...,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЦЕНТ...","ООО ""ЦЕНТР ПРЕДПРИНИМАТЕЛЬСТВА ""ДИНАСТИЯ""",7713291570,772501001,"115533, г Москва, р-н Нагатино-Садовники, Нага...",Москва,115533,45296571000,45918000000,57026232,Кацитадзе Владимир Владимирович,None,Общество с ограниченной ответственностью,ООО
4,ООО ЦЕНТР ПРЕДПРИНИМАТЕЛЬСТВА ДИНАСТИЯ,7713291570,772501001,ГОРОД МОСКВА НАБЕРЕЖНАЯ НАГАТИНСКАЯ ДОМ 10 КОР...,"ФИЛИАЛ ООО ""ЦЕНТР ПРЕДПРИНИМАТЕЛЬСТВА ""ДИНАСТИ...","ФИЛИАЛ ООО ""ЦЕНТР ПРЕДПРИНИМАТЕЛЬСТВА ""ДИНАСТИ...",7713291570,None,"115533, г Москва, р-н Нагатино-Садовники, Нага...",Москва,115533,None,None,None,None,None,Филиал юридического лица,Филиал
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,АО РОССИЙСКИЙ КОНЦЕРН ПО ПРОИЗВОДСТВУ ЭЛЕКТРИЧ...,7721632827,772101001,ГОРОД МОСКВА УЛИЦА ФЕРГАНСКАЯ 25,"ФИЛИАЛ АО ""КОНЦЕРН РОСЭНЕРГОАТОМ"" ""ОПЫТНО-ДЕМО...","ФИЛИАЛ АО ""КОНЦЕРН РОСЭНЕРГОАТОМ"" ""ОПЫТНО-ДЕМО...",7721632827,365143004,"396070, Воронежская обл, г Нововоронеж, ул Кур...",Нововоронеж,396070,None,None,None,None,None,Филиал юридического лица,Филиал
9990,АО РОССИЙСКИЙ КОНЦЕРН ПО ПРОИЗВОДСТВУ ЭЛЕКТРИЧ...,7721632827,772101001,ГОРОД МОСКВА УЛИЦА ФЕРГАНСКАЯ 25,"ФИЛИАЛ АО ""КОНЦЕРН РОСЭНЕРГОАТОМ"" ""ОПЫТНО-ДЕМО...","ФИЛИАЛ АО ""КОНЦЕРН РОСЭНЕРГОАТОМ"" ""ОПЫТНО-ДЕМО...",7721632827,472643003,"188540, Ленинградская обл, г Сосновый Бор",Сосновый Бор,188540,None,None,None,None,None,Филиал юридического лица,Филиал
9991,СОДНТ САДОВОДЧЕСКОЕ НЕКОММЕРЧЕСКОЕ ТОВАРИЩЕСТВ...,5027099677,502701001,ОБЛАСТЬ МОСКОВСКАЯ ГОРОД ЛЮБЕРЦЫ ДЕРЕВНЯ МАРУС...,"САДОВОДЧЕСКОЕ НЕКОММЕРЧЕСКОЕ ТОВАРИЩЕСТВО ""ФОР...","СНТ ""ФОРТУНА""",5027099677,502701001,"140050, Московская обл, г Люберцы, деревня Мар...",Люберцы,140050,46448000036,46748000136,39850022,Титов Владимир Викторович,None,Товарищество собственников недвижимости,ТСН
9992,ПК ПОТРЕБИТЕЛЬСКИЙ ЖИЛИЩНЫЙ КООПЕРАТИВ ИНДИВИД...,5030045583,775101001,Г. МОСКВА П. Первомайское Д. Ширяево ПР-Д ТРОИ...,ПОТРЕБИТЕЛЬСКИЙ ЖИЛИЩНЫЙ КООПЕРАТИВ ИНДИВИДУАЛ...,"ПЖКИЗ ""ИЗУМРУДНАЯ ДОЛИНА""",5030045583,775101001,"108807, г Москва, поселение Первомайское, Трои...",Москва,108807,45298572128,45955000000,13230687,Грищенко Елена Евгеньевна,None,Потребительский кооператив,ПК


In [96]:
pd.DataFrame(df_raw, 
             columns=['RBK_NAME', 'RBK_INN', 'RBK_KPP', 'RBK_ADDR', 'DD_FNAME', 'DD_SNAME', 
                'DD_INN', 'DD_KPP', 'DD_ADDR', 'DD_CITY', 'DD_POSTAL_CODE', 'DD_OKATO'
                , 'DD_OKTMO', 'DD_OKPO', 'DD_MANAGER_NAME', 'DD_PHONES', 'DD_OPF_FULL', 'DD_OPF_SHORT']).to_excel('../data/raw/df_companies_40000_50000.xlsx', index=False)

### Объединяем файлы в один dataframe

In [97]:
df1 = pd.read_excel('../data/raw/df_companies_0_10000.xlsx', engine='openpyxl')
df2 = pd.read_excel('../data/raw/df_companies_10000_20000.xlsx', engine='openpyxl')
df3 = pd.read_excel('../data/raw/df_companies_20000_30000.xlsx', engine='openpyxl')
df4 = pd.read_excel('../data/raw/df_companies_40000_50000.xlsx', engine='openpyxl')

In [98]:
df = pd.concat([df1, df2, df3, df4])

In [102]:
df.dropna(subset=['DD_FNAME']).to_excel('../data/raw/df_companies_whole_48528.xlsx', index=False)

In [159]:
df_dddd[df_dddd['DD_FNAME'].isnull()]

,RBK_NAME,RBK_INN,RBK_KPP,RBK_ADDR,DD_FNAME,DD_SNAME,DD_INN,DD_KPP,DD_ADDR,DD_CITY,DD_POSTAL_CODE,DD_OKATO,DD_OKTMO,DD_OKPO,DD_MANAGER_NAME,DD_PHONES,DD_OPF_FULL,DD_OPF_SHORT
67,ГБУ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ДИГОР...,1507002102,150701001,РЕСПУБЛИКА СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ РАЙОН ДИГО...,None,None,None,None,None,None,None,None,None,None,None,None,None,None
135,ГБУ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ДИГОР...,1507002102,150701001,РЕСПУБЛИКА СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ РАЙОН ДИГО...,None,None,None,None,None,None,None,None,None,None,None,None,None,None
183,АО КАЗАНСКОЕ ПАО ОРГАНИЧЕСКИЙ СИНТЕЗ,1658008723,165801001,РЕСПУБЛИКА ТАТАРСТАН ГОРОД КАЗАНЬ УЛИЦА БЕЛОМО...,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
pd.concat([df, df_dd, df_ddd, df_dddd]).dropna(subset=['DD_FNAME']).info()#.to_excel('../data/raw/df_companies_whole_56859.xlsx', index=False)

### Проверка данных по банкам и получение филиалов

In [321]:
# Берем для дальнейшей генерации файлы с пометкой unique

In [165]:
df_bank = pd.read_excel('../data/raw/BANKS.xlsx', engine='openpyxl')

In [179]:
df_bank[~df_bank['ИНН'].isna()]

# df_bank['ИНН'].astype(object)

,Название банка,Краткое название банка,ИНН,КПП,ОГРН,БИК,Адрес юрид.,Адрес факт.,Регистрац. номер,Действующий
0,"Акционерный коммерческий банк ""1Банк"" (публичн...","ПАО АКБ ""1Банк""",1.504030e+09,150201001.0,1021500000147,49033746.0,"362040, Республика Северная Осетия-Алания, г.В...","362040, Республика Северная Осетия-Алания, г.В...",2896,True
1,"Акционерный коммерческий банк ""Абсолют Банк"" (...","АКБ ""Абсолют Банк"" (ПАО)",7.736047e+09,770201001.0,1027700024560,44525976.0,"127051, г. Москва, Цветной бульвар, д. 18","127051, г. Москва, Цветной бульвар, д. 18",2306,False
2,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...","ПАО АКБ ""АВАНГАРД""",7.702021e+09,770501001.0,1027700367507,44525201.0,"115035, г. Москва, ул. Садовническая, д.12, с...","115035, г. Москва, ул. Садовническая, д.12, с...",2879,False
3,Общество с ограниченной ответственностью Банк ...,"ООО Банк ""Аверс""",1.653004e+09,165501001.0,1021600000993,49205774.0,"420111, г.Казань, ул.Мусы Джалиля, д.3","420111, г.Казань, ул.Мусы Джалиля, д.3",415,False
4,"Акционерное общество ""Автоградбанк""","АО ""Автоградбанк""",1.650072e+09,165001001.0,1021600000806,49205878.0,"423831, Республика Татарстан, г.Набережные Чел...","423831, Республика Татарстан, г.Набережные Чел...",1455,False
...,...,...,...,...,...,...,...,...,...,...
773,общество с ограниченной ответственностью небан...,"ООО НКО ""ЮМани""",7.750006e+09,770501001.0,1127711000031,44525444.0,"115035, город Москва, Садовническая улица, дом...","115035, город Москва, Садовническая улица, дом...",3510-К,False
774,"Общество с ограниченной ответственностью ""Банк...","ООО ""ЮМК банк""",2.312262e+09,231201001.0,1172375051936,40349830.0,"350080, г. Краснодар, ул. Уральская, 95","350080, г. Краснодар, ул. Уральская, 95",3495,False
775,"Коммерческий банк ""ЮНИКОР"" (открытое акционерн...","ОАО ""ЮНИКОРБАНК""",7.744001e+09,771701001.0,1027700035835,44525397.0,"129515, г. Москва, ул. 1-я Останкинская, д. 26","129515, г. Москва, ул. 1-я Останкинская, д. 26",2586,True
776,"Акционерное общество ""ЮниКредит Банк""",АО ЮниКредит Банк,7.710030e+09,770401001.0,1027739082106,44525545.0,"119034, г. Москва, Пречистенская набережная, д.9","119034, г. Москва, Пречистенская набережная, д.9",1,False


In [186]:
# df_bank.columns

df_bank[df_bank.index == 256].values[0]

array(['Закрытое акционерное общество "ИпоТек Банк"', 'ЗАО "ИпоТек Банк"',
       nan, nan, 1026300004499, 44525603.0,
       '107023, г. Москва, ул. Электрозаводская, дом 20, строение 11',
       '107023, г. Москва, ул. Электрозаводская, дом 20, строение 11',
       '2794', True], dtype=object)

In [205]:
def get_dadata_bank(company_info):
    """
    Запуск получения данных от dadata и
    объединение их в один массив с данными от РБК
    """
    raw = []
    apy_key='8bfbf82611b02577af0cc47f8b0ec64785243586'
    try:
        dd_res = get_verification_data(company_info[1], apy_key)
        if dd_res is not None:
            for c in dd_res:
                
                raw.append([company_info[0],company_info[1], company_info[2], company_info[3], company_info[4],
                             company_info[5], 
                             company_info[6], 
                             company_info[7], company_info[8], company_info[9], 
                         c.get('Full_Name') if c.get('Full_Name') is not None else c.get('Name1'),
                                        c.get('Short_Name') if c.get('Short_Name') is not None else c.get('Name1'),
                                        c.get('INN'),
                                        c.get('KPP'),
                                        c.get('Address'),
                                        c.get('City'),
                                        c.get('postal_code'),
                                        c.get('okato'),
                                        c.get('oktmo'),
                                        c.get('okpo'),
                                        c.get('manager_name'),
                                        c.get('phones'),
                                        c.get('opf_full'),
                                        c.get('opf_short'),
                                   ])
        else:
            raw.append([company_info[0],company_info[1], company_info[2], company_info[3], company_info[4],
                             company_info[5], 
                             company_info[6], 
                             company_info[7], company_info[8], company_info[9],  
                        None, None, None, None, None, None, None, None, None, None, None, None, None, None])
    except:
        raw.append([company_info[0],company_info[1], company_info[2], company_info[3], company_info[4],
                             company_info[5], 
                             company_info[6], 
                             company_info[7], company_info[8], company_info[9], 
                        None, None, None, None, None, None, None, None, None, None, None, None, None, None])
    return raw

In [209]:
%%time

#TODO: пересобрать None значения
raww = pool.map(get_dadata_bank, df_bank.values)

CPU times: user 27.1 s, sys: 1.07 s, total: 28.1 s
Wall time: 1min


In [210]:
raww

df_raw = []
for i in raww:
    for j in i:
        df_raw += [j]

In [212]:
pd.DataFrame(df_raw, 
                             columns=['CBR_NAME', 'CBR_FULL_NAME', 'CBR_INN', 'CBR_KPP', 'CBR_OGRN',
                        'CBR_BIK', 'CBR_URADDR', 'CBR_FACTADDR', 'CBR_REG_NUM','CBR_ACTIVE', 'DD_FNAME',
                'DD_SNAME', 
                'DD_INN', 'DD_KPP', 'DD_ADDR', 'DD_CITY', 'DD_POSTAL_CODE', 'DD_OKATO'
                , 'DD_OKTMO', 'DD_OKPO', 'DD_MANAGER_NAME', 'DD_PHONES', 'DD_OPF_FULL', 'DD_OPF_SHORT'] ).to_excel('../data/raw/df_banks_1439.xlsx', index=False)

In [316]:
df_bank = pd.read_excel('../data/raw/df_banks_1439.xlsx', engine='openpyxl')

In [320]:
df_bank.to_excel('../data/raw/df_banks_unique_1354.xlsx', index=False)

### Получение информации по ИП

In [218]:
df_ip1 = pd.read_excel('../data/raw/IP_ORG1.xlsx', engine='openpyxl')
df_ip2 = pd.read_excel('../data/raw/IP_ORG2.xlsx', engine='openpyxl')

In [220]:
df_ip = pd.concat([df_ip1, df_ip2])

In [223]:
df_ip.head()

,NAME,INN,OGRNIP,ADDRESS
0,Алексеева Марина Евгеньевна,732710183223,321732500003322,Ульяновская область
1,Агаев Сарраф Ханлар Оглы,301707421116,306301735400019,"Астраханская область, город Астрахань"
2,Акбасов Равиль Уапович,300401716505,305302133600025,"Астраханская область, село Озерное"
3,Абдиров Алексей Исмагулович,301506283547,307301708300027,"Астраханская область, город Астрахань"
4,Андрющенко Кристина Валерьевна,501807819555,305501810300019,"Московская область, город Королёв"


In [240]:
def get_dadata_ip(company_info):
    """
    Запуск получения данных от dadata и
    объединение их в один массив с данными от РБК
    """
    raw = []
    apy_key='aa24430c3c36b7bfe23c0e26830797fb296fbf6d'
    try:
        dd_res = get_verification_data(company_info[1], apy_key)
        if dd_res is not None:
            for c in dd_res:
#                 columns=['RBK_NAME', 'RBK_INN', 'RBK_KPP', 'RBK_ADDR', 'DD_FNAME', 'DD_SNAME', 
#                 'DD_INN', 'DD_KPP', 'DD_ADDR', 'DD_CITY', 'DD_POSTAL_CODE', 'DD_OKATO'
#                 , 'DD_OKTMO', 'DD_OKPO', 'DD_MANAGER_NAME', 'DD_PHONES', 'DD_OPF_FULL', 'DD_OPF_SHORT'] 
                raw.append([company_info[0], company_info[1], None, company_info[3], #company_info[3],
                         c.get('Full_Name') if c.get('Full_Name') is not None else c.get('Name1'),
                                        c.get('Short_Name') if c.get('Short_Name') is not None else c.get('Name1'),
                                        c.get('INN'),
                                        c.get('KPP'),
                                        c.get('Address'),
                                        c.get('City'),
                                        c.get('postal_code'),
                                        c.get('okato'),
                                        c.get('oktmo'),
                                        c.get('okpo'),
                                        c.get('manager_name'),
                                        c.get('phones'),
                                        c.get('opf_full'),
                                        c.get('opf_short'),
                                   ])
        else:
            raw.append([company_info[0],company_info[1], company_info[2], company_info[3], 
                        None, None, None, None, None, None, None, None, None, None, None, None, None, None])
    except:
        raw.append([company_info[0],company_info[1], company_info[2], company_info[3], 
                        None, None, None, None, None, None, None, None, None, None, None, None, None, None])
    return raw

In [228]:
df_ip.values[0]

array(['Алексеева Марина Евгеньевна', 732710183223, 321732500003322,
       'Ульяновская область'], dtype=object)

In [236]:
get_dadata_ip(df_ip.values[0])

[['Алексеева Марина Евгеньевна',
  None,
  732710183223,
  'Ульяновская область',
  'Индивидуальный предприниматель Алексеева Марина Евгеньевна',
  'ИП Алексеева Марина Евгеньевна',
  '732710183223',
  None,
  '432000, Ульяновская обл, г Ульяновск',
  'Ульяновск',
  '432000',
  '73401365000',
  None,
  '2005072575',
  None,
  None,
  'Индивидуальный предприниматель',
  'ИП'],
 ['Алексеева Марина Евгеньевна',
  None,
  732710183223,
  'Ульяновская область',
  'Индивидуальный предприниматель Алексеева Марина Евгеньевна',
  'ИП Алексеева Марина Евгеньевна',
  '732710183223',
  None,
  '432000, Ульяновская обл, г Ульяновск',
  'Ульяновск',
  '432000',
  '73401365000',
  None,
  '0185261000',
  None,
  None,
  'Индивидуальный предприниматель',
  'ИП']]

In [244]:
%%time

#TODO: пересобрать None значения
raww = pool.map(get_dadata_ip, df_ip.values)

CPU times: user 4min 41s, sys: 8.77 s, total: 4min 50s
Wall time: 8min 20s


In [245]:
raww

df_raw = []
for i in raww:
    for j in i:
        df_raw += [j]

In [248]:
df_ip = pd.DataFrame(df_raw, 
             columns=['RBK_NAME', 'RBK_INN', 'RBK_KPP', 'RBK_ADDR', 'DD_FNAME', 'DD_SNAME', 
                'DD_INN', 'DD_KPP', 'DD_ADDR', 'DD_CITY', 'DD_POSTAL_CODE', 'DD_OKATO'
                , 'DD_OKTMO', 'DD_OKPO', 'DD_MANAGER_NAME', 'DD_PHONES', 'DD_OPF_FULL', 'DD_OPF_SHORT'])#.to_excel('../data/raw/df_ip_10202.xlsx', index=False)

### Объединяем Организации и ИП

In [322]:
# Берем для дальнейшей генерации файлы с пометкой unique

In [103]:
df_org = pd.read_excel('../data/raw/df_companies_whole_48528.xlsx', engine='openpyxl')

In [104]:
df_ip = pd.read_excel('../data/raw/df_ip_10202.xlsx', engine='openpyxl')

In [107]:
# Добавляем "ИП" к РБК наименованию
df_ip['RBK_NAME'] = df_ip['RBK_NAME'].apply(lambda x: "ИП "+x)

In [108]:
df_org = pd.concat([df_org, df_ip])#.to_excel('../data/raw/df_companies_whole_org_ip_67061.xlsx', engine='openpyxl')

In [110]:
def del_dot_zero(st):
    return str(st).rstrip('.0')

In [111]:
df_org['DD_INN'] = df_org['DD_INN'].apply(del_dot_zero)
df_org['DD_KPP'] = df_org['DD_KPP'].apply(del_dot_zero)
df_org['DD_POSTAL_CODE'] = df_org['DD_POSTAL_CODE'].apply(del_dot_zero)
df_org['DD_OKATO'] = df_org['DD_OKATO'].apply(del_dot_zero)
df_org['DD_OKTMO'] = df_org['DD_OKTMO'].apply(del_dot_zero)
df_org['DD_OKPO'] = df_org['DD_OKPO'].apply(del_dot_zero)
df_org['RBK_INN'] = df_org['RBK_INN'].apply(del_dot_zero)
df_org['RBK_KPP'] = df_org['RBK_KPP'].apply(del_dot_zero)

In [112]:
df_org = df_org[['RBK_NAME', 'RBK_INN', 'RBK_KPP', 'RBK_ADDR', 'DD_FNAME',
       'DD_SNAME', 'DD_INN', 'DD_KPP', 'DD_ADDR', 'DD_CITY', 'DD_POSTAL_CODE',
       'DD_OKATO', 'DD_OKTMO', 'DD_OKPO', 'DD_MANAGER_NAME', 'DD_PHONES',
       'DD_OPF_FULL', 'DD_OPF_SHORT']]

In [113]:
df_org_unique = df_org.drop_duplicates(subset='DD_FNAME', keep='last')

In [114]:
df_org_unique[df_org_unique['DD_INN'] == '774307763']

,RBK_NAME,RBK_INN,RBK_KPP,RBK_ADDR,DD_FNAME,DD_SNAME,DD_INN,DD_KPP,DD_ADDR,DD_CITY,DD_POSTAL_CODE,DD_OKATO,DD_OKTMO,DD_OKPO,DD_MANAGER_NAME,DD_PHONES,DD_OPF_FULL,DD_OPF_SHORT
48468,БФ БФ ВСЕМИРНЫЙ ИНСТИТУТ ХРИСТИАНСКИХ ИССЛЕДОВ...,774307763,774301001,ГОРОД МОСКВА УЛИЦА ПЕТРОЗАВОДСКАЯ 11/3,"БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""ВСЕМИРНЫЙ ИНСТИТУТ ХРИ...","БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""ВСЕМИРНЫЙ ИНСТИТУТ ХРИ...",774307763,774301001,"125502, г Москва, р-н Ховрино, ул Петрозаводск...",Москва,125502,45277595,45347,712654,"Долгорукий-Симанский (Быков), Рюрикович Монома...",NaN,Благотворительный фонд,БФ
48469,БФ БФ ВСЕМИРНЫЙ ИНСТИТУТ ХРИСТИАНСКИХ ИССЛЕДОВ...,774307763,774301001,ГОРОД МОСКВА УЛИЦА ПЕТРОЗАВОДСКАЯ 11/3,"ФИЛИАЛ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""ВСЕМИРНЫЙ ИНСТИ...","ФИЛИАЛ БЛАГОТВОРИТЕЛЬНЫЙ ФОНД ""ВСЕМИРНЫЙ ИНСТИ...",774307763,nan,"125502, г Москва, р-н Ховрино, ул Петрозаводск...",Москва,125502,nan,nan,nan,NaN,NaN,Филиал юридического лица,Филиал


In [115]:
df_org_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42798 entries, 60 to 10201
Data columns (total 18 columns):
RBK_NAME           42798 non-null object
RBK_INN            42798 non-null object
RBK_KPP            42798 non-null object
RBK_ADDR           42798 non-null object
DD_FNAME           42798 non-null object
DD_SNAME           42798 non-null object
DD_INN             42798 non-null object
DD_KPP             42798 non-null object
DD_ADDR            42798 non-null object
DD_CITY            36232 non-null object
DD_POSTAL_CODE     42798 non-null object
DD_OKATO           42798 non-null object
DD_OKTMO           42798 non-null object
DD_OKPO            42798 non-null object
DD_MANAGER_NAME    29494 non-null object
DD_PHONES          0 non-null float64
DD_OPF_FULL        42798 non-null object
DD_OPF_SHORT       42798 non-null object
dtypes: float64(1), object(17)
memory usage: 6.2+ MB


In [116]:
df_org_unique[['RBK_NAME', 'RBK_INN', 'RBK_KPP', 'RBK_ADDR', 'DD_FNAME',
       'DD_SNAME', 'DD_INN', 'DD_KPP', 'DD_ADDR', 'DD_CITY', 'DD_POSTAL_CODE',
       'DD_OKATO', 'DD_OKTMO', 'DD_OKPO', 'DD_MANAGER_NAME', 'DD_PHONES',
       'DD_OPF_FULL', 'DD_OPF_SHORT']].to_excel('../data/raw/df_companies_whole_unique_org_ip_42798.xlsx', index=False, engine='openpyxl')